In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from gensim.models import KeyedVectors
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,GlobalAveragePooling1D,concatenate

In [3]:
# Path to the pretrained GloVe model file
glove_path = '/data/home/vvaibhav/AI/glove.6B.200d.txt'

# Load the GloVe model using gensim
glove_model = KeyedVectors.load_word2vec_format(glove_path, binary=False)

In [4]:
# Load the CSV file without header
df = pd.read_csv('ClassificationDataset-train1.csv', header=None)
df_test = pd.read_csv('ClassificationDataset-valid1.csv', header=None)
print(df.shape)
# print(df.iloc[1,0])
# Get x_train and y_train
x_train_neg = df.iloc[1:,0]
x_train_pos = df.iloc[1:,1]
print(x_train_neg.shape, x_train_pos.shape)

x_train_neg_array = x_train_neg.to_numpy()
x_train_pos_array = x_train_pos.to_numpy()
x_train_array = np.concatenate([x_train_neg_array, x_train_pos_array])
print(x_train_array.shape)
zeros = np.zeros(df.shape[0]-1)
print(zeros.shape)
ones = np.ones(df.shape[0]-1)
print(ones.shape)
y_train_array = np.concatenate([zeros,ones])
print(y_train_array.shape)

#similarly get x_test and y_test
x_test_neg = df_test.iloc[1:,0]
x_test_pos = df_test.iloc[1:,1]
x_test_neg = x_test_neg.dropna()
x_test_pos = x_test_pos.dropna()
x_test_neg_array = x_test_neg.to_numpy()
x_test_pos_array = x_test_pos.to_numpy()
x_test_array = np.concatenate([x_test_neg_array, x_test_pos_array])

zeros = np.zeros(x_test_neg.shape)
ones = np.ones(x_test_pos.shape)
y_test_array = np.concatenate([zeros,ones])

(6333, 6)
(6332,) (6332,)
(12664,)
(6332,)
(6332,)
(12664,)


In [5]:
print("negative train samples : ", x_train_neg_array.shape)
print("positive train samples : ", x_train_pos_array.shape)
print("complete train shape :", x_train_array.size,'*',y_train_array.shape)

negative train samples :  (6332,)
positive train samples :  (6332,)
complete train shape : 12664 * (12664,)


In [6]:
# Convert text data to embeddings-based representations
def text_to_embeddings(text, embeddings):
    embeddings_list = [embeddings[word] if word in embeddings else np.zeros(embeddings.vector_size) for word in text]
    #print(len(embeddings_list[0]))
    return np.mean(embeddings_list, axis=0)

In [7]:
import tensorflow as tf

# Convert text data to embeddings-based representations
def text_to_embeddings(text, embeddings):
    if len(text) == 0 :
       return np.zeros(embeddings.vector_size)
    embeddings_list = [embeddings[word] if word in embeddings else np.zeros(embeddings.vector_size) for word in text]
    #print(len(embeddings_list[0]))
    return np.mean(embeddings_list, axis=0)

X_train_embeddings = []
for text in x_train_array:
  text = text.split()
  embeddings = text_to_embeddings(text, glove_model)
  if embeddings.shape != (200, ) :
     print("hello", text)
  X_train_embeddings.append(embeddings)

X_test_embeddings = []
for text in x_test_array:
  text = text.split()
  embeddings = text_to_embeddings(text, glove_model)
  if embeddings.shape != (200, ) :
     print("hello", text)
  X_test_embeddings.append(embeddings)

print(len(X_train_embeddings), len(X_test_embeddings))

X_train_embeddings = np.array(X_train_embeddings, dtype=object)
X_test_embeddings = np.array(X_test_embeddings, dtype=object)



12664 5334


In [8]:
X_train_embeddings = X_train_embeddings.astype(np.float64)
X_test_embeddings = X_test_embeddings.astype(np.float64)

In [9]:
print(X_train_embeddings.shape)
print(type(X_train_embeddings))
print(type(X_train_embeddings[0][0]))

(12664, 200)
<class 'numpy.ndarray'>
<class 'numpy.float64'>


In [10]:
input_size = X_train_embeddings.shape[1]
num_classes = len(np.unique(y_train_array))


In [11]:
model = Sequential()

model.add(Input(shape = (input_size)))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model
batch_size = 256
num_epochs = 50

model.fit(X_train_embeddings, y_train_array, batch_size=batch_size, epochs=num_epochs)

Epoch 1/50
50/50 [==============================] - 3s 5ms/step - loss: 0.5041 - accuracy: 0.7920
Epoch 2/50
50/50 [==============================] - 0s 6ms/step - loss: 0.3644 - accuracy: 0.8488
Epoch 3/50
50/50 [==============================] - 0s 7ms/step - loss: 0.3411 - accuracy: 0.8583
Epoch 4/50
50/50 [==============================] - 0s 7ms/step - loss: 0.3281 - accuracy: 0.8620
Epoch 5/50
50/50 [==============================] - 0s 6ms/step - loss: 0.3195 - accuracy: 0.8662
Epoch 6/50
50/50 [==============================] - 0s 5ms/step - loss: 0.3107 - accuracy: 0.8683
Epoch 7/50
50/50 [==============================] - 0s 6ms/step - loss: 0.3036 - accuracy: 0.8715
Epoch 8/50
50/50 [==============================] - 0s 5ms/step - loss: 0.2963 - accuracy: 0.8751
Epoch 9/50
50/50 [==============================] - 0s 5ms/step - loss: 0.2919 - accuracy: 0.8759
Epoch 10/50
50/50 [==============================] - 0s 5ms/step - loss: 0.2848 - accuracy: 0.8814
Epoch 11/50
50/50 [

In [13]:
y_pred = model.predict(X_test_embeddings)
y_pred_labels = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test_array, y_pred_labels)
f1_weighted = f1_score(y_test_array, y_pred_labels, average='micro')
f1_macro = f1_score(y_test_array, y_pred_labels, average='macro')
# "average='micro'" to calculate F1-scores that consider all classes equally.

print("Test accuracy:", accuracy)
print("Micro F1 score:", f1_weighted)
print("Macro F1 score:", f1_macro)

Test accuracy: 0.8254593175853019
Micro F1 score: 0.8254593175853019
Macro F1 score: 0.8241572189246968
